# Trabalho Computacional 3. Rede Convolucional e Transfer Learning

## 1. Introdução e Base de Dados

Neste trabalho usaremos uma rede convolucional pré-treinada e a aplicaremos em um problema novo. Também experimentaremos com a
divisão da base em treinamento, validação e teste, e usaremos o conjunto de validação para a técnica "early stopping", na tentativa de controlar o sobre-ajuste.

A base de dados é a [CIFAR10](https://www.cs.toronto.edu/~kriz/cifar.html). Ela contém 60000 imagens 32x32 coloridas (3 canais) das seguintes categorias de objetos: ‘airplane’, ‘automobile’, ‘bird’, ‘cat’, ‘deer’, ‘dog’, ‘frog’, ‘horse’, ‘ship’, ‘truck’.

Ela pode ser baixada com o código abaixo.

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

In [2]:
class CIFAR10():  #@save    
    def __init__(self, root, resize=(224, 224)):    
        trans = transforms.Compose([transforms.Resize(resize),
                                    transforms.ToTensor(),
                                    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
        self.train = torchvision.datasets.CIFAR10(
            root=root, train=True, transform=trans, download=True)
        # use 20% of training data for validation
        train_set_size = int(len(self.train) * 0.8)
        valid_set_size = len(self.train) - train_set_size
         # split the train set into two
        seed = torch.Generator().manual_seed(42)
        self.train, self.val = torch.utils.data.random_split(self.train, [train_set_size, valid_set_size], generator=seed)
        self.test = torchvision.datasets.CIFAR10(
            root=root, train=False, transform=trans, download=True)
        
dataset = CIFAR10(root="./data/")

train_dataloader = torch.utils.data.DataLoader(dataset.train, batch_size=64, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(dataset.val, batch_size=64, shuffle=False)
test_dataloader = torch.utils.data.DataLoader(dataset.test, batch_size=64, shuffle=False)

print(f"Number of training examples: {len(dataset.train)}")
print(f"Number of validation examples: {len(dataset.val)}")
print(f"Number of test examples: {len(dataset.test)}")

Number of training examples: 40000
Number of validation examples: 10000
Number of test examples: 10000


Observe como foi feita a separação de 10.000 exemplos do conjunto de treinamento original para serem o conjunto de validação. Dessa forma, temos ao final 40.000 exemplos para treinamento, 10.000 exemplos para validação e 10.000 exemplos para teste, com os seus respectivos `DataLoader`'s instanciados.

Também redimensionamos as imagens para 224x224pixels, já preparando o dado para a posterior aplicação na rede convolucional.

## 2. Treinando um MLP

Use esta base de dados para treinar um Perceptron Multicamadas, como feito no trabalho anterior com a base MNIST. Escolha um MLP com 2 camadas escondidas. Não perca muito tempo variando a arquitetura porque este problema é difícil sem o uso de convoluções e o resultado não será totalmente satisfatório.

Você pode usar este código, baseado na biblioteca [Pytorch Lightning](https://lightning.ai/docs/pytorch/stable/) como base para definição da rede:

In [ ]:
%pip install pytorch-lightning
import pytorch_lightning as pl
import torch.nn as nn
from torchmetrics.functional import accuracy

# The model is passed as an argument to the `LightModel` class.
class LightModel(pl.LightningModule):
	def __init__(self,model,lr=1e-5):
		super().__init__()
		self.model = model
		self.lr = lr
	def training_step(self, batch):
		X, y = batch
		y_hat = self.model(X)
		loss = nn.functional.cross_entropy(y_hat, y)
		self.log("train_loss", loss)
		return loss
	def validation_step(self, batch):
		X, y = batch
		y_hat = self.model(X)
		loss = nn.functional.cross_entropy(y_hat, y)
		self.log("val_loss", loss)
		return loss
	def test_step(self, batch):
		X, y = batch
		y_hat = self.model(X)
		preds = torch.argmax(y_hat, dim=1)
		acc = accuracy(preds, y, task="multiclass", num_classes=10)
		self.log("test_acc", acc)
		loss = nn.functional.cross_entropy(y_hat, y)		
		self.log("test_loss", loss)		
	def configure_optimizers(self):
		optimizer = torch.optim.Adam(self.parameters(), self.lr)
		return optimizer


arch = nn.Sequential(
			nn.Flatten(),
			nn.Linear(3*224*224,?),
			nn.ReLU(),
			nn.Linear(?,?),
			nn.ReLU(),
			nn.Linear(?,10)	
	)

mlp = LightModel(arch)

SyntaxError: invalid syntax (44956469.py, line 39)

Observe que as imagens são achatadas (transformadas em vetor). Substitua as interrogações pelo tamanho desejado das camadas
escondidas.

Neste problema vamos verificar o fenômeno do sobreajuste, e vamos tentar equilibrá-lo pela técnica de parada prematura de treinamento (early-stopping).
Por isso foi necessário, a partir dos dados de treinamento, fazer uma nova separação para validação. Quando a função custo (`loss`) no conjunto de validação não diminui num dado número de épocas (o parâmetro `patience`), o treinamento é interrompido. Este trecho de código pode ser útil:

In [ ]:
from pytorch_lightning.callbacks import EarlyStopping
from pytorch_lightning import Trainer

early_stopping = EarlyStopping(
    monitor='val_loss',  # metric to monitor
    patience=5,          # epochs with no improvement after which training will stop
    mode='min',          # mode for min loss; 'max' if maximizing metric
    min_delta=0.001      # minimum change to qualify as an improvement
)

trainer = Trainer(callbacks=[early_stopping], max_epochs=50)
trainer.fit(model=mlp, train_dataloaders=train_dataloader, val_dataloaders=val_dataloader)

Os parâmetros dados são sugestões. Você agora pode testar o seu modelo, por exemplo, com:

In [ ]:
# Evaluate the model on the test dataset
trainer.test(model=mlp, dataloaders=test_dataloader)

Mais uma vez, procure realizar ajustes, mas não espere um bom desempenho. Como dissemos, é um problema complexo de classificação de
imagem, e é difícil fazer o MLP funcionar sozinho. Precisamos de um pré-processamento com base em uma rede convolucional.

## 3. Uso da rede VGG16 pré-treinada

Lembre-se que a rede VGG usa como bloco básico cascata de convoluções com filtros 3x3, com "padding" para que a imagem não seja
diminuída, seguida de um "max pooling" reduzindo imagens pela metade. O número de mapas vai aumentando e seu tamanho vai diminuindo
ao longo de suas 16 camadas. Este é um modelo gigantesco e o treinamento com recursos computacionais modestos levaria dias ou
semanas, se é que fosse possível.

No entanto, vamos aproveitar uma característica central das grandes redes convolucionais. Elas podem ser usadas como pré-processamento
fixo das imagens, mesmo em um novo problema (lembre-se, a rede VGG original foi treinada na base ImageNet, que tem muitas categorias de
imagens).

O código abaixo realiza o download do modelo treinado e configura os seus parâmetros como não ajustáveis.

In [ ]:
from torchvision.models import vgg16
vgg16_model = vgg16(weights="DEFAULT", progress=True)

for param in vgg16_model.parameters():
	param.requires_grad = False

print(vgg16_model)

Agora, modifique o bloco de classificação (`vgg16_model.classifier`) da VGG16 de forma que fique semelhante ao MLP anterior, para trabalhar com somente 10 classes. Especifique uma camada `Flatten` (o MLP espera um vetor de entradas), e duas camadas densas. Elas não precisam ser muito grandes. Experimente com 50 e 20 neurônios, respectivamente, ou algo próximo.

Volte a treinar e testar o modelo (pode usar as classes `LightModel` e `Trainer`). Mesmo sem efetivamente treinar toda a rede VGG (somente os parâmetros do bloco de classificação), ainda temos que passar os dados por ela a cada passo, e o treinamento é um tanto lento. Mas desta vez o problema deve ser resolvido satisfatoriamente.

## 4. Extras (opcionais)

### 4.1. Procure usar outra(s) redes convolucionais como base.
### 4.2. No lugar de "early stopping", experimente com regularização L1 e L2, e "dropout".